In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold


from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

# pip install xgboost
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import permutation_importance

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

In [ ]:
data = pd.read_csv(f'PATH/t-DS3.csv.gz',compression='gzip',lineterminator='\n');

In [ ]:
# result = pd.concat([data[data["class"]=="bot"], data[data["class"]=="human"].sample(41333)])
# result = result.sample(frac=1).reset_index(drop=True)
# result.to_csv(f'PATH/DS3.csv.gz', compression='gzip',index=False)

In [ ]:
data.head(25)

,listed_count,statuses_count,favourites_count,profile_background_tile,profile_background_image_url_https,friends_count,has_extended_profile,followers_count,default_profile,profile_use_background_image,default_profile_image,profile_banner_url,geo_enabled,verified,entities,diff_days,name_length,screen_name_length,description_length,num_digits_in_name,num_digits_in_screen_name,tweet_freq,followers_growth_rate,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,Name_entropy,Screen_name_entropy,Screen_name_freq,Name_similarity,Name_freq,_url,_location,class
0,26,10189,9239,0,1,522,1,222,0,1,0,0,1,0,0,3446,1,14,0,0,1,2.956761,0.064423,0.151480,2.681080,0.007545,0.064423,-0.000000,0.206878,0.846154,0.000000,1.000000,0,1,human
1,1,15751,1651,0,1,227,1,183,0,1,0,1,1,0,1,4342,10,10,53,0,0,3.627591,0.042146,0.052280,0.380240,0.000230,0.042146,0.284644,0.232193,1.000000,0.500000,1.000000,1,0,human
2,4,178107,40218,0,1,100,1,144,0,0,0,1,1,0,1,1804,13,8,147,0,0,98.728936,0.079823,0.055432,22.293792,0.002217,0.079823,0.216544,0.343750,1.000000,0.095238,0.750000,1,1,bot
3,8,354666,7321,1,1,450,1,475,0,1,0,1,1,0,0,2596,25,11,89,0,0,136.620185,0.182974,0.173344,2.820108,0.003082,0.182974,0.165346,0.264907,0.800000,0.000000,0.916667,0,1,bot
4,9,80970,33990,0,1,9975,0,9636,1,1,0,1,0,0,1,4285,31,7,96,0,0,18.896149,2.248775,2.327888,7.932322,0.002100,2.248775,0.127024,0.360234,1.000000,0.263158,0.966667,1,1,bot
5,16,34040,52,0,1,3,0,3932,0,1,0,1,0,0,1,3779,10,10,120,0,0,9.007674,1.040487,0.000794,0.013760,0.004234,1.040487,0.272193,0.272193,0.777778,0.800000,0.777778,1,1,bot
6,0,2924,9270,0,1,526,1,145,0,0,0,1,1,0,0,4388,16,9,135,0,0,0.666363,0.033045,0.119872,2.112580,0.000000,0.033045,0.226562,0.327523,1.000000,0.080000,1.000000,0,1,human
7,0,5069,3019,0,1,253,1,60,1,1,0,1,1,0,0,3398,12,15,120,0,2,1.491760,0.017657,0.074456,0.888464,0.000000,0.017657,0.251838,0.212660,1.000000,0.740741,1.000000,0,1,bot
8,0,41678,1761,0,1,595,0,129,1,1,0,1,0,0,0,3216,35,15,30,0,0,12.959577,0.040112,0.185012,0.547575,0.000000,0.040112,0.124094,0.112818,0.500000,0.080000,1.000000,0,1,bot
9,1,807,1076,0,1,484,0,95,1,1,0,1,1,0,1,2750,12,8,155,0,2,0.293455,0.034545,0.176000,0.391273,0.000364,0.034545,0.232707,0.343750,1.000000,0.200000,1.000000,1,1,human


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

In [ ]:
def returnSelectedFeature(model,data,return_feature=15):

  feature_data=data.copy()
  feature_data['class'] = feature_data['class'].map({"bot": 1, "human": 0});

  selected_columns=feature_data.columns.difference(["class"])

  model.fit(feature_data[selected_columns], feature_data['class'])

  if type(model).__name__=='KNeighborsClassifier':
    importance = permutation_importance(model, feature_data[selected_columns], feature_data['class'], scoring='accuracy').importances_mean
  else:
    importance= model.feature_importances_

  feats = {} 
  for feature, importance in zip(selected_columns, importance):
      feats[feature] = importance

  return pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'order'}).sort_values(by='order')[-return_feature:].index.tolist()
def Normalizing(data,execption=["class"]):
  methods={
      "MinMaxScaler":MinMaxScaler(feature_range=(0, 1)),
      "Normalizer":Normalizer(),
      "Binarizer":Binarizer(threshold=0.0),
      "RobustScaler":RobustScaler(quantile_range=(25, 75)),
      "yeoJohnson":PowerTransformer(method="yeo-johnson"),
      "boxCox":PowerTransformer(method="box-cox"),
      "MaxAbsScaler":MaxAbsScaler(),
      "uniform":QuantileTransformer(output_distribution="uniform"),
      "normal":QuantileTransformer(output_distribution="normal"),
      "StandardScaler":StandardScaler(),
  }
  selected_columns=data.columns.difference(execption)
  result={}

  for key, value in methods.items():
    data_sample=data.copy()
    if key=="boxCox" and (data_sample[selected_columns] <= 0).values.any():
      continue
    data_sample[selected_columns] = value.fit_transform(data_sample[selected_columns])
    result[key]=data_sample

  return result
def SingleNormalizing(data,key,execption=["class"]):
  methods={
      "MinMaxScaler":MinMaxScaler(feature_range=(0, 1)),
      "Normalizer":Normalizer(),
      "Binarizer":Binarizer(threshold=0.0),
      "RobustScaler":RobustScaler(quantile_range=(25, 75)),
      "yeoJohnson":PowerTransformer(method="yeo-johnson"),
      "boxCox":PowerTransformer(method="box-cox"),
      "MaxAbsScaler":MaxAbsScaler(),
      "uniform":QuantileTransformer(output_distribution="uniform"),
      "normal":QuantileTransformer(output_distribution="normal"),
      "StandardScaler":StandardScaler(),
  }
  selected_columns=data.columns.difference(execption)
  data_sample=data.copy()
  data_sample[selected_columns] = methods[key].fit_transform(data_sample[selected_columns])

  return data_sample

In [ ]:
data = pd.read_csv(f'PATH/DS1.csv.gz',compression='gzip',lineterminator='\n');
data=data.sample(frac=1).reset_index(drop=True)

In [ ]:
randomforest_classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
datasets=["DS4","DS5","DS6","DS7","DS12","DS13","DS14","DS15"]
featureSelectionMethod={
      "all":"",
      "decisionTree":DecisionTreeClassifier(),
      "randomForest":RandomForestClassifier(n_estimators=100),
      "XGBClassifier":XGBClassifier(),
      # "kNeighbors":KNeighborsClassifier(n_neighbors=3)
}

testDataSet={}
classifer_accuracy_result=pd.DataFrame(columns=["DS","Normalizer","FeatureSelected","b-precision","b-recall","b-f1-score","b-total","h-precision","h-recall","h-f1-score","h-total","a-accuracy"])

for index,dataset in enumerate(datasets):
  testDataSet[dataset] = pd.read_csv(f'PATH/{dataset}.csv.gz',compression='gzip',lineterminator='\n');


for key,value in Normalizing(data).items():
  for keyFeature,valueFeature in featureSelectionMethod.items():

    if keyFeature=="all":
        selected_columns=value.columns.difference(["class"]).tolist()
    else: 
      selected_columns=returnSelectedFeature(valueFeature,value,10)

    randomforest_classifier.fit(value[selected_columns], value['class'])
    
    for keyTest,dataTest in testDataSet.items():

        normalizedTest=SingleNormalizing(dataTest,key).copy()
        classifer_accuracy_row=[keyTest,key,keyFeature]


        for index_2,(keyShow,valueShow) in enumerate(classification_report(normalizedTest["class"], randomforest_classifier.predict(normalizedTest[selected_columns]),output_dict=True).items()):
          if keyShow in ["bot","human"]:
            classifer_accuracy_row.extend([*valueShow.values()])
          elif keyShow=="accuracy":
            classifer_accuracy_row.append(valueShow)
        print(key,keyFeature,keyTest,classifer_accuracy_row)
        classifer_accuracy_result.loc[len(classifer_accuracy_result)]=classifer_accuracy_row
classifer_accuracy_result.to_csv(f'PATH/DT_2022-06-01.csv.gz', compression='gzip',index=False)

StandardScaler randomForest DS4 ['DS4', 'StandardScaler', 'randomForest', 0.5309597523219814, 0.9716713881019831, 0.6866866866866868, 353, 0.7872340425531915, 0.10882352941176471, 0.19121447028423777, 340, 0.5483405483405484]
StandardScaler randomForest DS5 ['DS5', 'StandardScaler', 'randomForest', 0.33221250840618693, 0.1391157420444945, 0.19610956728860657, 7102, 0.40652300524170065, 0.6783284742468416, 0.5083758193736344, 6174, 0.38987646881590843]
StandardScaler randomForest DS6 ['DS6', 'StandardScaler', 'randomForest', 0.4346590909090909, 0.8571428571428571, 0.5768143261074458, 1071, 0.5887096774193549, 0.15498938428874734, 0.2453781512605042, 1413, 0.4577294685990338]
StandardScaler randomForest DS7 ['DS7', 'StandardScaler', 'randomForest', 0.6802633365006109, 0.23613532488338124, 0.3505771248688353, 42446, 0.09443078985588202, 0.41782006920415227, 0.15404592673592124, 8092, 0.26522616644900865]
StandardScaler randomForest DS12 ['DS12', 'StandardScaler', 'randomForest', 0.8962060

In [ ]:
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
datasets=["DS4","DS5","DS6","DS7","DS12","DS13","DS14","DS15"]
featureSelectionMethod={
      "all":"",
      "decisionTree":DecisionTreeClassifier(),
      "randomForest":RandomForestClassifier(n_estimators=100),
      "XGBClassifier":XGBClassifier(),
      # "kNeighbors":KNeighborsClassifier(n_neighbors=3)
}

testDataSet={}
classifer_accuracy_result=pd.DataFrame(columns=["DS","Normalizer","FeatureSelected","b-precision","b-recall","b-f1-score","b-total","h-precision","h-recall","h-f1-score","h-total","a-accuracy"])

for index,dataset in enumerate(datasets):
  testDataSet[dataset] = pd.read_csv(f'PATH/{dataset}.csv.gz',compression='gzip',lineterminator='\n');


for key,value in Normalizing(data).items():
  for keyFeature,valueFeature in featureSelectionMethod.items():

    if keyFeature=="all":
        selected_columns=value.columns.difference(["class"]).tolist()
    else: 
      selected_columns=returnSelectedFeature(valueFeature,value,15)

    classifier.fit(value[selected_columns], value['class'])
    
    for keyTest,dataTest in testDataSet.items():

        normalizedTest=SingleNormalizing(dataTest,key).copy()
        classifer_accuracy_row=[keyTest,key,keyFeature]


        for index_2,(keyShow,valueShow) in enumerate(classification_report(normalizedTest["class"], classifier.predict(normalizedTest[selected_columns]),output_dict=True).items()):
          if keyShow in ["bot","human"]:
            classifer_accuracy_row.extend([*valueShow.values()])
          elif keyShow=="accuracy":
            classifer_accuracy_row.append(valueShow)
        # print(key,keyFeature,keyTest)
        classifer_accuracy_result.loc[len(classifer_accuracy_result)]=classifer_accuracy_row
classifer_accuracy_result.to_csv(f'PATH/r2022-05-31.csv.gz', compression='gzip',index=False)

In [ ]:
from sklearn.svm import SVC 

classifier = SVC()

datasets=["DS4","DS5","DS6","DS7","DS12","DS13","DS14","DS15"]
featureSelectionMethod={
      "all":"",
      "decisionTree":DecisionTreeClassifier(),
      "randomForest":RandomForestClassifier(n_estimators=100),
      "XGBClassifier":XGBClassifier(),
      # "kNeighbors":KNeighborsClassifier(n_neighbors=3)
}

testDataSet={}
classifer_accuracy_result=pd.DataFrame(columns=["DS","Normalizer","FeatureSelected","b-precision","b-recall","b-f1-score","b-total","h-precision","h-recall","h-f1-score","h-total","a-accuracy"])

for index,dataset in enumerate(datasets):
  testDataSet[dataset] = pd.read_csv(f'PATH/{dataset}.csv.gz',compression='gzip',lineterminator='\n');


for key,value in Normalizing(data).items():
  for keyFeature,valueFeature in featureSelectionMethod.items():

    if keyFeature=="all":
        selected_columns=value.columns.difference(["class"]).tolist()
    else: 
      selected_columns=returnSelectedFeature(valueFeature,value,15)

    classifier.fit(value[selected_columns], value['class'])
    
    for keyTest,dataTest in testDataSet.items():

        normalizedTest=SingleNormalizing(dataTest,key).copy()
        classifer_accuracy_row=[keyTest,key,keyFeature]


        for index_2,(keyShow,valueShow) in enumerate(classification_report(normalizedTest["class"], classifier.predict(normalizedTest[selected_columns]),output_dict=True).items()):
          if keyShow in ["bot","human"]:
            classifer_accuracy_row.extend([*valueShow.values()])
          elif keyShow=="accuracy":
            classifer_accuracy_row.append(valueShow)
        print(key,keyFeature,keyTest)
        classifer_accuracy_result.loc[len(classifer_accuracy_result)]=classifer_accuracy_row
classifer_accuracy_result.to_csv(f'PATH/SVM_15_2022-06-01.csv.gz', compression='gzip',index=False)